Introduction à Rust

# Séance 3 – `Option` et `Result`

[Pierre-Antoine Champin](http://champin.net/)

Département Info Doua – [IUT Lyon 1](http://iut.univ-lyon1.fr/)

## Le type `Option` 🦀

### Motivation

* En Java, une fonction dont le type de retour est `MyObject` peut en général retourner une instance de `MyObjet` ou `null`. (Idem en C avec une fonction qui retourne un pointeur).

* Ceci est une entorse au système de typage, car `null` n'est *pas* du type `MyObject`:

    + il n'a aucune des méthodes de cette classe,
    + et toute tentative de l'utiliser comme tel déclenchera une erreur (`NullPointerException`).


* En Rust, une fonction dont le type de retour est `T` doit retourner une valeur de ce type.

* Mais il y a pourtant des cas où on souhaite pouvoir retourner « un `T` ou rien du tout »... Exemples :

    + la méthode `pop()` du type `Vec<T>` retire le dernier élément du vecteur *et retourne sa valeur*,
      mais ne fait rien (et ne retourne rien) si le vecteur était déjà vide ;

    + la méthode `get(i)` du type `[T]` retourne l'élément d'indice *i* s'il existe,
      ou rien si *i* est trop grand. 

* Le type de retour de ces méthodes est `Option<T>` ; les valeurs de ce type peuvent être :

    * `None`, qui représente l'absence de valeur, ou
    * `Some(t)`, où *t* est une valeur de type `T`.

### `Option<T>` en action

In [2]:
fn safe_sub(a: u32, b: u32) -> Option<u32> {
    if a >= b {
        Some(a-b)
    } else {
        None
    }
}

safe_sub(5,2)

Some(3)

In [3]:
match safe_sub(5, 2) {
    None => println!("Le résultat serait négatif."),
    Some(r) => println!("Le résultat est {}.", r),
};

Le résultat est 3.


In [4]:
// rappel: la clause match est une expression
let c = match safe_sub(2, 5) {
    None => '-',
    Some(0) => '0',
    Some(_) => '+',
};
c

'-'

### Les méthodes `expect` et `unwrap`

Lorsqu'on ne souhaite pas traiter le cas ou une option vaut `None`,
on peut utiliser sa méthode `expect(message)` :

* pour `Some(t)`, elle retourne `t`;
* pour `None`, elle interrompt le programme (panique) en affichant le message passé en paramètre.

In [5]:
let v = vec![1,1,2,3,5];
let val: i32 = v.pop().expect("v ne devrait pas être vide");

Error: cannot borrow `v` as mutable, as it is not declared as mutable

La méthode `unwrap` fonctionne de la même manière, mais ne prend pas de paramètre.
Elle affichera un message par défaut en cas de panique.

## Le type `Result` 🦀

### Motivation

* Dans certaines situations, le fait de ne pas retourner une valeur du type attendu est un comportement *anormal*.

    + On souhaite alors pouvoir connaître la *raison* de cette anomalie.
    
* Pour ces situations, Rust fournit le type `Result<T,E>`, dont les valeurs peuvent être :

    + `Ok(t)` où *t* est de type `T`, et qui représente le résultat normal de l'opération, ou
    + `Err(e)` où *e* est de type `E`, et représente une anomalie dont *e* est une explication.

### `Result<T, E>` en action

In [6]:
fn vitesse(distance: f64, temps: f64) -> Result<f64, &'static str> {
    if distance < 0.0 {
        Err("distance ne peut pas être négative")
    } else if temps < 0.0 {
        Err("temps ne peut pas être négatif")
    } else if temps == 0.0 {
        Err("le temps est nul, la vitesse serait infinie")
    } else {
        Ok(distance/temps)
    }
}

vitesse(70.0, 0.8)

Ok(87.5)

In [7]:
match vitesse(70.0, 0.8) {
    Err(msg) => eprintln!("Erreur: {}", msg),
    Ok(v) => if v>130.0 {
            println!("Vous allez beaucoup trop vite !");
        } else {
            println!("Votre vitesse est de {} km/h", v);
        }
}

Votre vitesse est de 87.5 km/h


()

NB: le type `Result<T, E>` a lui aussi une méthode `expect(message)` et une méthode `unwrap()`.

In [8]:
let v: f64 = vitesse(70.0, 0.8).unwrap();

### Faire remonter une erreur

In [9]:
fn vitesse_moyenne(d1: f64, t1: f64, d2: f64, t2: f64) -> Result<f32, &'static str> {
    let v1 = match vitesse(d1, t1) {
        Ok(v) => v,
        Err(err) => { return Err(err); }
    };
    let v2 = match vitesse(d2, t2) {
        Ok(v) => v,
        Err(err) => { return Err(err); }
    };
    Ok(((v1+v2)/2.0) as f32)
}

In [10]:
fn vitesse_moyenne2(d1: f64, t1: f64, d2: f64, t2: f64) -> Result<f32, &'static str> {
    let v1 = vitesse(d1, t1)?;
    let v2 = vitesse(d2, t2)?;
    Ok(((v1+v2)/2.0) as f32)
}